# Machine-Readable Data Formats

## Recommendations and Best Practices for Biodiversity Informatics

### ***Giuditta Parolini, Data Scientist, Museum für Naturkunde Berlin***

---

# Table of Contents
* [Introduction](#intro)
* [Section 1: The trouble with non-machine-readable data](#trouble)
    * [1.1: Data published as a PDF file](#pdf)
    * [1.2: Data published as a DOCX file](#docx)
* [Section 2: Machine-readable data formats for tabular data](#tabular)
    * [2.1: CSV, TSV](#csv)
    * [2.2: TXT](#txt)
    * [2.3: XML](#xml)
    * [2.4: JSON](#json)
    * [2.5: RDF](#rdf)
    * [2.6: Parquet](#parquet)

---

In [1]:
# imports
import os
import json
import pprint
from pathlib import Path
from docx.api import Document
from io import StringIO
from timeit import default_timer as timer
from lxml import etree
from tabula import read_pdf
from rdflib import Graph
import pandas as pd

---

## Introduction <a class="anchor" id="intro"></a>

**This [Jupyter notebook](https://jupyter.org) provides practical examples that illustrate the main points discussed in the Guide on machine-readable data.**
<br>

It allows readers to see examples of the machine-readable data formats suggested, realise the challenges posed by data that are not machine-readable, and experience the pitfalls that can cause the generation of invalid files even when using machine-readable data formats like CSV. The notebook also describes how unstructured data, like digital images or other media, can be approached to provide, at least, a few pieces of machine-readable information.
<br>

Throughout the notebook, examples will be illustrated using the dataset ***Mounted Specimens of the Historical Bird Collection at the Museum für Naturkunde Berlin*** (DOI: [10.7479/wwqn-gd04](https://doi.org/10.7479/wwqn-gd04)) and modifications of it. The dataset contains metadata for over 13000 images of mounted bird specimens belonging to the bird collection of the museum. The mounted specimens have been systematically photographed and their images and related metadata are distributed under a [CC0 Public Domain Dedication](https://creativecommons.org/publicdomain/zero/1.0/deed.en) license. ***A copy of the original dataset is available in the GitHub repository as dataset.csv***.
<br>

**All data files used in this notebook are hosted in the data folder or can be downloaded from the Internet**.


In [2]:
cwd = Path.cwd() # path to the directory hosting the Jupyter notebook
data_path = os.path.join(cwd, "data") # path to the data folder

---

## 1: The trouble with non-machine-readable data <a class="anchor" id="trouble"></a>
As mentioned in the guide Introduction, PDF and DOCX files are human-readable, but not really machine-readable and extracting data from them is a challenging and error-prone exercise. An example of this will be demonstrated in this section using a PDF and a DOCX documents containing an extract of the bird collection dataset and its metadata. The content of both files is the same. It will be shown how extracting the data, which would be immediately available in a CSV file, and saving them in a machine-readable format can become a lengthy and troublesome business.



### 1.1: Data published as a PDF file <a class="anchor" id="pdf"></a>

The PDF document here examined has a total of four pages containing a single data table preceded by a text description with the dataset metadata.
<br>
<br>
![Here](pdf_overview.png)

The Python library [tabula-py](https://tabula-py.readthedocs.io/en/latest/index.html) allows to extract tables from a PDF document and save them in a machine-readable format like CSV or JSON. Reading the pdf file with tabula-py requires only a line of code, but the result needs further adjustments. The multiple line heading in the table is misinterpreted at reading time and generates two extra-rows that need to be removed manually. In addition, as the table is displayed on several pages, tabula-py interprets each page as a different table with the result that the table sections not on the first page have data rows interpreted as headings (see below).

In [3]:
# Reading the pdf file with tabula-py
df = read_pdf(os.path.join(data_path, "PDF_doc_example.pdf"), pages='all')

In [4]:
# The multiple line heading is misinterpreted at reading time (rows 0 ad 1).
df[0].head(10)

,catalogue_id,title (scientific,class,family,genus,species
0,NaN,name or,NaN,NaN,NaN,NaN
1,NaN,common name),NaN,NaN,NaN,NaN
2,ZMB_AVES_2000-14765,Rhea americana,Aves,Rheidae,Rhea,Rhea americana
3,ZMB_AVES_2000-14765,Rhea americana,Aves,Rheidae,Rhea,Rhea americana
4,ZMB_AVES_2000-31350,Struthio camelus,Aves,Struthionidae,Struthio,Struthio camelus
5,ZMB_AVES_2000-31350,Struthio camelus,Aves,Struthionidae,Struthio,Struthio camelus
6,NaN,Struthio,NaN,NaN,NaN,Struthio
7,ZMB_AVES_2000-31351,molybdophanes,Aves,Struthionidae,Struthio,molybdophanes
8,ZMB_AVES_2000-31795,Rhea americana,Aves,Rheidae,Rhea,Rhea americana
9,ZMB_AVES_2000-32382,Rhea pennata,Aves,Rheidae,Rhea,Rhea pennata


In [5]:
# A new table with misleading headings starts on page 2 (and similarly for the other pages)
df[1]

,ZMB_Aves_10,Alectoris chukar,Aves,Phasianidae,Alectoris,Alectoris chukar.1
0,ZMB_Aves_100,Calidris maritima,Aves,Scolopacidae,Calidris,Calidris maritima
1,ZMB_Aves_100,Calidris maritima,Aves,Scolopacidae,Calidris,Calidris maritima
2,ZMB_Aves_10007,Psittacula\ralexandri\ralexandri,Aves,Psittacidae,Psittacula,Psittacula\ralexandri
3,ZMB_Aves_10007,Psittacula\ralexandri\ralexandri,Aves,Psittacidae,Psittacula,Psittacula\ralexandri
4,ZMB_Aves_10051,Alisterus\rscapularis\rscapularis,Aves,Psittacidae,Alisterus,Alisterus\rscapularis
5,ZMB_Aves_10051,Alisterus\rscapularis\rscapularis,Aves,Psittacidae,Alisterus,Alisterus\rscapularis
6,ZMB_Aves_1006,Falco peregrinus,Aves,Falconidae,Falco,Falco\rpelegrinoides
7,ZMB_Aves_1006,Falco peregrinus,Aves,Falconidae,Falco,Falco\rpelegrinoides
8,ZMB_Aves_10066,Trichoglossus\rmoluccanus\rmoluccanus,Aves,Psittacidae,Trichoglossus,Trichoglossus\rmoluccanus
9,ZMB_Aves_10066,Trichoglossus\rmoluccanus\rmoluccanus,Aves,Psittacidae,Trichoglossus,Trichoglossus\rmoluccanus


The mis-intepretation of the table on different pages can be removed in tabula-py by changing the default value for the argument multiple_tables to False. The heading set on multiple lines can be read appropriately using the argument lattice set to True (tabula-py will not interpret line breaks as new rows)

In [6]:
df_single_table = read_pdf(os.path.join(data_path, "PDF_doc_example.pdf"),
                           pages='all', multiple_tables=False, lattice=True )
df_single_table[0]

,catalogue_id,title (scientific\rname or\rcommon name),class,family,genus,species
0,ZMB_AVES_2000-14765,Rhea americana,Aves,Rheidae,Rhea,Rhea americana
1,ZMB_AVES_2000-14765,Rhea americana,Aves,Rheidae,Rhea,Rhea americana
2,ZMB_AVES_2000-31350,Struthio camelus,Aves,Struthionidae,Struthio,Struthio camelus
3,ZMB_AVES_2000-31350,Struthio camelus,Aves,Struthionidae,Struthio,Struthio camelus
4,ZMB_AVES_2000-31351,Struthio\rmolybdophanes,Aves,Struthionidae,Struthio,Struthio\rmolybdophanes
...,...,...,...,...,...,...
94,ZMB_Aves_1047,Falco rusticolus,Aves,Falconidae,Falco,Falco rusticolus
95,ZMB_Aves_1048,Falco rusticolus,Aves,Falconidae,Falco,Falco rusticolus
96,ZMB_Aves_1048,Falco rusticolus,Aves,Falconidae,Falco,Falco rusticolus
97,ZMB_Aves_105,Calandrella\rbrachydactyla\rbrachydactyla,Aves,Alaudidae,Calandrella,Calandrella\rbrachydactyla


However, if the Pandas dataframe so obtained is turned into a machine-readable CSV without further preprocessing, the result will be a malformed file, as the carriage return signs (\r) will be misinterpreted in the generation of the CSV (see right below).

In [7]:
# Saving the dataframe to CSV
df_single_table[0].to_csv(os.path.join(data_path, "dataset_from_pdf.csv"), index=False)

This is how the CSV file extracted using tabula-py looks likE when opened with a text editor:
<br>
![Here](invalid_csv_from_pdf.png)

### 1.2: Data published as a DOCX file <a class="anchor" id="docx"></a>

[Python-docx](https://python-docx.readthedocs.io/en/latest/api/document.html) is a library for creating DOCX files using the Python programming language. As the library is able to create a DOCX file, it is also useful to extract content from them.

In [8]:
# All the document content can be extracted as a Python generator object
content = Document(os.path.join(data_path,'DOCX_doc_example.docx')).iter_inner_content()
content


<generator object BlockItemContainer.iter_inner_content at 0x1191ece50>

In [9]:
# The generator object can be unpacked in a list to see all the components in the DOCX file.
# In our case, we have the text paragraphs and the data table
docx_list = [el for el in content]
docx_list 

In [10]:
# It is easier to consider text paragraphs separated from the data table,
# so we remove the table from the list
docx_list.pop(17)
docx_list

In [11]:
# Now the text content can be joined and printed.
content = '\n'.join([p.text for p in docx_list])
print(content)

Dataset title
Mounted Specimens of the Historical Bird Collection at the Museum für Naturkunde Berlin

Creator
MfN Digitization

License
CC0 1.0 Creative Commons Public Domain Dedication

Dataset Description
The dataset contains metadata for over 13000 images of mounted bird specimens belonging to the bird collection of the Museum für Naturkunde Berlin (MfN). The mounted specimens were mostly collected in the 19th century and have now been systematically photographed. For large part of the mounted specimens the full taxonomy is available. When this is not the case, the specimens are identified with their German common names. In the dataset there are birds from the well-known families of pigeons (Columbidae), parrots (Psittacidae) and pheasants (Phasianidae) as well as duck birds (Anatidae) and exotic hummingbirds (Trochilidae). Some rare specimens such as the quetzal (Pharomachrus mocinno) are also available in this dataset. 

Keywords
Birds, mounted specimens, Museum für Naturkunde Be

In [12]:
# One can also save this information to a TXT file for later re-use
with open(os.path.join(data_path, "text_extracted.txt"), "w") as text_file:
    text_file.write(content)

In [13]:
# The table data can be extracted using a for loop and then saved into a pandas dataframe.
# The dataframe can then be saved as a CSV file. 
# Code inspired by Stackoverflow
# (https://stackoverflow.com/questions/46618718/python-docx-to-extract-table-from-word-docx)

start = timer()

document = Document(os.path.join(data_path, "DOCX_doc_example.docx"))
table = document.tables[0]

data = []

keys = None
for i, row in enumerate(table.rows):
    text = (cell.text for cell in row.cells)

    if i == 0:
        keys = tuple(text)
        continue
    row_data = dict(zip(keys, text))
    data.append(row_data)

df = pd.DataFrame(data)

end = timer()

The data table extracted is the following: 

In [14]:
df

,catalogue_id,title (scientific name or common name),class,family,genus,species
0,ZMB_AVES_2000-14765,Rhea americana,Aves,Rheidae,Rhea,Rhea americana
1,ZMB_AVES_2000-31350,Struthio camelus,Aves,Struthionidae,Struthio,Struthio camelus
2,ZMB_AVES_2000-31351,Struthio molybdophanes,Aves,Struthionidae,Struthio,Struthio molybdophanes
3,ZMB_AVES_2000-31795,Rhea americana,Aves,Rheidae,Rhea,Rhea americana
4,ZMB_AVES_2000-32382,Rhea pennata,Aves,Rheidae,Rhea,Rhea pennata
5,ZMB_AVES_2000-34658,Struthio camelus,Aves,Struthionidae,Struthio,Struthio camelus
6,ZMB_AVES_2000-34923,Rhea americana,Aves,Rheidae,Rhea,Rhea americana
7,ZMB_AVES_2000-632,Struthio camelus,Aves,Struthionidae,Struthio,Struthio camelus
8,ZMB_AVES_2000-752,Casuarius unappendiculatus,Aves,Casuariidae,Casuarius,Casuarius unappendiculatus
9,ZMB_AVES_2000-8516,Struthio camelus,Aves,Struthionidae,Struthio,Struthio camelus


The dataframe is well-formed and there is no issue in the data extraction, however the data have been extracted looping over the table element in the DOCX file. Loops are inefficient in Python and while in this case there is no real time issue due to the very limited table size, problems would immediately emerge when real-scale datasets with thousands of rows and tens of columns need to be extracted.

The time required by the loop to run can be computed using Python [timeit](https://docs.python.org/3/library/timeit.html) library (see code cell above) and the result is:

In [15]:
# Process time for extracting the data table from the DOCX file
print(str(end - start) + "s", "required to extract 312 data cells from a DOCX file") #computed in seconds

0.053925904037896544s required to extract 312 data cells from a DOCX file


In [16]:
# By contrast, reading the entire dataset in machine-readable format
read_start = timer()
dataset_read_from_csv = pd.read_csv(os.path.join(data_path, "dataset.csv"))
read_end = timer()
print(str(read_end - read_start) + "s", "required to read in 199320 data cells (=13288rows × 15columns) from a CSV file") #computed in seconds

0.15845057897968218s required to read in 199320 data cells (=13288rows × 15columns) from a CSV file


Although times remain manageable for both solutions in this case, with datasets having millions and billions of data cells the data extraction si going to become more and more time expensive making the user regret not to have the data directly available in a machine-readable format like CSV.

---

## 2: Machine-readable data formats for tabular data <a class="anchor" id="tabular"></a>



### 2.1: CSV, TSV <a class="anchor" id="csv"></a>

CSV files might not be the solution to all data problems, but they are definitely handy for delivering tabular data in a machine-readable format. For datasets with up to 1 Million data rows they should be the first data format considered. 

In [17]:
# With the Python Pandas programming library reading a CSV file only takes a line code 
df_csv_comma_sep = pd.read_csv(os.path.join(data_path,"dataset.csv"))
df_csv_comma_sep.head(2) #display the first two rows of the dataset 

,catalogue_id,key,scientific_name,title (scientific name or common name),class,family,genus,species,subspecies,collections,creation_year,absolute_url,copyright,license,authors
0,ZMB_AVES_2000-14765,7d1264cca2e93009237e,Rhea americana,Rhea americana,Aves,Rheidae,Rhea,Rhea americana,NaN,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
1,ZMB_AVES_2000-14765,46e9071723ddbababd06,Rhea americana,Rhea americana,Aves,Rheidae,Rhea,Rhea americana,NaN,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization


In [18]:
# A copy of the original dataset has been saved using the semicolon as a delimiter
df_csv_semicolon_sep = pd.read_csv(os.path.join(data_path,"dataset_semicolon.csv"))
df_csv_semicolon_sep.head(2) 

,catalogue_id;key;scientific_name;title (scientific name or common name);class;family;genus;species;subspecies;collections;creation_year;absolute_url;copyright;license;authors
0,ZMB_AVES_2000-14765;7d1264cca2e93009237e;Rhea ...
1,ZMB_AVES_2000-14765;46e9071723ddbababd06;Rhea ...


As the expectation is to have the comma as a separator, the result is wrong, but it can be easily corrected. It is enough to specify the correct separator when reading in the data to import the dataset without issues.

In [19]:
df_csv_semicolon_sep = pd.read_csv(os.path.join(data_path,"dataset_semicolon.csv"), sep=";")
df_csv_semicolon_sep.head(2) # Now the dataframe is correctly read by Pandas

,catalogue_id,key,scientific_name,title (scientific name or common name),class,family,genus,species,subspecies,collections,creation_year,absolute_url,copyright,license,authors
0,ZMB_AVES_2000-14765,7d1264cca2e93009237e,Rhea americana,Rhea americana,Aves,Rheidae,Rhea,Rhea americana,NaN,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,MfN Berlin; https://ror.org/052d1a351,CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
1,ZMB_AVES_2000-14765,46e9071723ddbababd06,Rhea americana,Rhea americana,Aves,Rheidae,Rhea,Rhea americana,NaN,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,MfN Berlin; https://ror.org/052d1a351,CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization


Similarly for reading the dataset in tsv format.

In [20]:
df_csv_tab_sep = pd.read_csv(os.path.join(data_path,"dataset.tsv"), sep="\t")
df_csv_tab_sep.head(2) # Once the correct separator is specified the TSV file is read correctly.

,catalogue_id,key,scientific_name,title (scientific name or common name),class,family,genus,species,subspecies,collections,creation_year,absolute_url,copyright,license,authors
0,ZMB_AVES_2000-14765,7d1264cca2e93009237e,Rhea americana,Rhea americana,Aves,Rheidae,Rhea,Rhea americana,NaN,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,MfN Berlin; https://ror.org/052d1a351,CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
1,ZMB_AVES_2000-14765,46e9071723ddbababd06,Rhea americana,Rhea americana,Aves,Rheidae,Rhea,Rhea americana,NaN,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,MfN Berlin; https://ror.org/052d1a351,CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization


The Pandas library allows also to read in files (and even workbooks)in XLSX format. In this case, as the data table has been created properly, the dataset is also read by Pandas without issues. However, Python Pandas takes longer to read in an XLSX file compared to a CSV file with potential performance issues for large datasets.

In [21]:
df_xlsx = pd.read_excel(os.path.join(data_path,"dataset.xlsx"))
df_xlsx.head(2)

,catalogue_id,key,scientific_name,title (scientific name or common name),class,family,genus,species,subspecies,collections,creation_year,absolute_url,copyright,license,authors
0,ZMB_AVES_2000-14765,7d1264cca2e93009237e,Rhea americana,Rhea americana,Aves,Rheidae,Rhea,Rhea americana,NaN,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
1,ZMB_AVES_2000-14765,46e9071723ddbababd06,Rhea americana,Rhea americana,Aves,Rheidae,Rhea,Rhea americana,NaN,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization


WARNING: The chances to create non-machine-readable files are much higher when working with spreadsheet software like Excel rather than dealing directly with the CSV data format.
Here an example of the birds dataset formatted in Excel with added descriptions, empty cells, ect.
![Here](invalid_dataset.png)

The invalid dataset above is read in without error warnings, but recovering the data requires a lengthy clean up
of all the empty cells and of the cells that contain the dataset description.

In [22]:
df_xlsx = pd.read_excel(os.path.join(data_path,"dataset_invalid_format.xlsx"))
df_xlsx

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,NaN,Dataset title,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Mounted Specimens of the Historical Bird Colle...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Dataset Description/Abstract,NaN,NaN,Data license,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,The dataset contains metadata for over 13000 i...,The images are available under a free license ...,NaN,CC0 Public Domain Dedication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13293,NaN,NaN,NaN,ZMB_Aves_9985,ca892e3d44beef9ca4dc,Psittacula krameri borealis,Psittacula krameri borealis,Aves,Psittacidae,Psittacula,Psittacula krameri,Psittacula krameri borealis,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
13294,NaN,NaN,NaN,ZMB_Aves_999,e76c936cca035eb8557d,Falco biarmicus tanypterus,Falco biarmicus tanypterus,Aves,Falconidae,Falco,Falco biarmicus,Falco biarmicus tanypterus,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
13295,NaN,NaN,NaN,ZMB_Aves_999,e5672b9636cec6d995e8,Falco biarmicus tanypterus,Falco biarmicus tanypterus,Aves,Falconidae,Falco,Falco biarmicus,Falco biarmicus tanypterus,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
13296,NaN,NaN,NaN,ZMB_Aves_9994,6ac8193d5913aa55de96,Psittacula longicauda longicauda,Psittacula longicauda longicauda,Aves,Psittacidae,Psittacula,Psittacula longicauda,Psittacula longicauda longicauda,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization


A CSV file is valid even when it does not have column headers. When the headers are missing, however, the user need to check that the data analysis software is correctly interpreting the first row as a data row and not as table headings.

In [23]:
df_csv_no_heading = pd.read_csv(os.path.join(data_path,"dataset_no_heading.csv"), header=None) # header=None added to avoid the first
                                                                        # row being considered the table header
df_csv_no_heading.head(2) # When the headers are missing, Python Pandas just identifies the data columns
                          # with an integer number.

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,ZMB_AVES_2000-14765,7d1264cca2e93009237e,Rhea americana,Rhea americana,Aves,Rheidae,Rhea,Rhea americana,NaN,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
1,ZMB_AVES_2000-14765,46e9071723ddbababd06,Rhea americana,Rhea americana,Aves,Rheidae,Rhea,Rhea americana,NaN,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization


### 2.2: TXT <a class="anchor" id="txt"></a>

TXT files should be the preferred machine-readable format for unstructured and not annotated text that needs to be further analysed/mined. As an example, let's consider the text extracted from the DOCX file in [Section 1.2](#docx).

In [24]:
# Reading the file content
with open(os.path.join(data_path,"text_extracted.txt"), "r") as f:
    content = f.read()

print(content)

Dataset title
Mounted Specimens of the Historical Bird Collection at the Museum für Naturkunde Berlin

Creator
MfN Digitization

License
CC0 1.0 Creative Commons Public Domain Dedication

Dataset Description
The dataset contains metadata for over 13000 images of mounted bird specimens belonging to the bird collection of the Museum für Naturkunde Berlin (MfN). The mounted specimens were mostly collected in the 19th century and have now been systematically photographed. For large part of the mounted specimens the full taxonomy is available. When this is not the case, the specimens are identified with their German common names. In the dataset there are birds from the well-known families of pigeons (Columbidae), parrots (Psittacidae) and pheasants (Phasianidae) as well as duck birds (Anatidae) and exotic hummingbirds (Trochilidae). Some rare specimens such as the quetzal (Pharomachrus mocinno) are also available in this dataset. 

Keywords
Birds, mounted specimens, Museum für Naturkunde Be

In [25]:
type(content) # The extracted text is treated as a string

str

Let's now focus on the dataset description. As we are working with plain text, there is no machine-readable indicator of where this section of text starts and finishes. We can only extract it relying on the knowledge we have of the original file structure, i.e., the dataset description is the set of words that follow the heading "Dataset Description" and ends before the following heading "Keywords". A possible way to extract the required text is to use the headings to split the text and then select the relevant part.

In [26]:

partition1 = "Dataset Description" #First partition heading
words = content.partition(partition1) #First split at the section heading
words_after_heading1 = content.split(partition1, 1)[1] #Selecting only the text after the first partition
partition2 = "Keywords" #Second partition heading
words_before_heading2 = words_after_heading1.split(partition2, 1)[0] #Selecting only the text after the first partition
                                                                    # and before the second partition heading
print(words_before_heading2) # Checking that the variable contains the required text (It does)


The dataset contains metadata for over 13000 images of mounted bird specimens belonging to the bird collection of the Museum für Naturkunde Berlin (MfN). The mounted specimens were mostly collected in the 19th century and have now been systematically photographed. For large part of the mounted specimens the full taxonomy is available. When this is not the case, the specimens are identified with their German common names. In the dataset there are birds from the well-known families of pigeons (Columbidae), parrots (Psittacidae) and pheasants (Phasianidae) as well as duck birds (Anatidae) and exotic hummingbirds (Trochilidae). Some rare specimens such as the quetzal (Pharomachrus mocinno) are also available in this dataset. 




If the text document had been provided with XML tags for the headings, it would have been much easier to extract the portion of text related to the dataset description. For instance, if there is available an XML tagged file like text_extracted.xml where the headings and the text body following the heading are tagged, it is possible to do as follows:

In [27]:
# Reading in the xml file
with open(os.path.join(data_path,"text_extracted.xml")) as f:
    xml = f.read()
xml # checking that the file has been read properly


'<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<doc:data xmlns:doc="https://example.com">\n  <doc:row>\n    <doc:heading>Dataset Title</doc:heading>\n    <doc:body>Mounted Specimens of the Historical Bird Collection at the Museum für Naturkunde Berlin</doc:body>\n  </doc:row>\n   <doc:row>\n    <doc:heading>Creator</doc:heading>\n    <doc:body>MfN Digitization</doc:body>\n  </doc:row>\n   <doc:row>\n    <doc:heading>License</doc:heading>\n    <doc:body>CC0 1.0 Creative Commons Public Domain Dedication</doc:body>\n  </doc:row>\n  <doc:row>\n    <doc:heading>Dataset Description</doc:heading>\n    <doc:body>The dataset contains metadata for over 13000 images of mounted bird specimens belonging to the bird collection of the Museum für Naturkunde Berlin (MfN). The mounted specimens were mostly collected in the 19th century and have now been systematically photographed. For large part of the mounted specimens the full taxonomy is available. When this is not the case, the specimens are identifi

In [28]:
# The text can be automatically transformed in a dataframe using the XML tags
df = pd.read_xml(StringIO(xml))
df

,heading,body
0,Dataset Title,Mounted Specimens of the Historical Bird Colle...
1,Creator,MfN Digitization
2,License,CC0 1.0 Creative Commons Public Domain Dedication
3,Dataset Description,The dataset contains metadata for over 13000 i...
4,Keywords,"Birds, mounted specimens, Museum für Naturkund..."
5,Table 1,"Shortened version of dataset, “Mounted Specime..."


In [29]:
# The dataset description is immediately available as a string in this case
df["body"][df.heading == "Dataset Description"].values[0]

'The dataset contains metadata for over 13000 images of mounted bird specimens belonging to the bird collection of the Museum für Naturkunde Berlin (MfN). The mounted specimens were mostly collected in the 19th century and have now been systematically photographed. For large part of the mounted specimens the full taxonomy is available. When this is not the case, the specimens are identified with their German common names. In the dataset there are birds from the well-known families of pigeons (Columbidae), parrots (Psittacidae) and pheasants (Phasianidae) as well as duck birds (Anatidae) and exotic hummingbirds (Trochilidae). Some rare specimens such as the quetzal (Pharomachrus mocinno) are also available in this dataset.'

### 2.3: XML <a class="anchor" id="xml"></a>

The content of a XML (eXtensible Markup Language) file is a combination of tags, which logically structure the content, and proper data. A version of the birds dataset in XML format will be used to illustrate the main features of the XML file format and the added features, like validation and comments, that it offers compared to a CSV file.

In [30]:
# Reading and printing to screen the XML data 
with open (os.path.join(data_path, "dataset.xml")) as f:
    content = f.readlines()

for line in content[0:10]: # limited the printed sample to 10 rows
    print(line)

<?xml version='1.0' encoding='utf-8'?>

<data>

  <row>

    <catalogue_id>ZMB_AVES_2000-14765</catalogue_id>

    <key>7d1264cca2e93009237e</key>

    <scientific_name>Rhea americana</scientific_name>

    <title>Rhea americana</title>

    <class>Aves</class>

    <family>Rheidae</family>

    <genus>Rhea</genus>



In [31]:
# The XML data can also be read as a dataframe, not differently than the CSV file
df_xml = pd.read_xml(os.path.join(data_path,"dataset.xml")) #The dataset column that had white spaces in the heading
                                    # needed renaming. The XML parser would otherwise throw an error
df_xml

,catalogue_id,key,scientific_name,title,class,family,genus,species,subspecies,collections,creation_year,absolute_url,copyright,license,authors
0,ZMB_AVES_2000-14765,7d1264cca2e93009237e,Rhea americana,Rhea americana,Aves,Rheidae,Rhea,Rhea americana,None,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
1,ZMB_AVES_2000-14765,46e9071723ddbababd06,Rhea americana,Rhea americana,Aves,Rheidae,Rhea,Rhea americana,None,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
2,ZMB_AVES_2000-31350,cd50680edb26a356d7f1,Struthio camelus,Struthio camelus,Aves,Struthionidae,Struthio,Struthio camelus,None,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
3,ZMB_AVES_2000-31350,1af8bb8651519838f87c,Struthio camelus,Struthio camelus,Aves,Struthionidae,Struthio,Struthio camelus,None,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
4,ZMB_AVES_2000-31351,b2d8c2a150d82453d35c,Struthio molybdophanes,Struthio molybdophanes,Aves,Struthionidae,Struthio,Struthio molybdophanes,None,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13283,ZMB_Aves_9985,ca892e3d44beef9ca4dc,Psittacula krameri borealis,Psittacula krameri borealis,Aves,Psittacidae,Psittacula,Psittacula krameri,Psittacula krameri borealis,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
13284,ZMB_Aves_999,e76c936cca035eb8557d,Falco biarmicus tanypterus,Falco biarmicus tanypterus,Aves,Falconidae,Falco,Falco biarmicus,Falco biarmicus tanypterus,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
13285,ZMB_Aves_999,e5672b9636cec6d995e8,Falco biarmicus tanypterus,Falco biarmicus tanypterus,Aves,Falconidae,Falco,Falco biarmicus,Falco biarmicus tanypterus,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
13286,ZMB_Aves_9994,6ac8193d5913aa55de96,Psittacula longicauda longicauda,Psittacula longicauda longicauda,Aves,Psittacidae,Psittacula,Psittacula longicauda,Psittacula longicauda longicauda,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization


The XML data file has the following **prolog**.


In [32]:
content[0]

"<?xml version='1.0' encoding='utf-8'?>\n"

In this case the prolog only consists of the **XML declaration**. An integral part of this declaration is the document encoding.


In many cases, the XML declaration will be followed by a **Document type declaration** that specifies the root element of the document and point to a **Document type definition**, i.e., markup declarations that provide a grammar for a class of XML documents. This is for instance a public XHTML document type declaration: 

In [33]:
#<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
# "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

When encountering this declaration, a parser tool will refer to the public document type definition linked and interpret the XML file accordingly.

The root note in the XML version of the bird dataset is data

In [34]:
content[1]

'<data>\n'

The elements of the XML file are the **rows**. Each row contains all the values for the columns in the original dataset. Each column value is enclosed within tags.

In [35]:
content[2:19] # The first row element in the XML dataset

['  <row>\n',
 '    <catalogue_id>ZMB_AVES_2000-14765</catalogue_id>\n',
 '    <key>7d1264cca2e93009237e</key>\n',
 '    <scientific_name>Rhea americana</scientific_name>\n',
 '    <title>Rhea americana</title>\n',
 '    <class>Aves</class>\n',
 '    <family>Rheidae</family>\n',
 '    <genus>Rhea</genus>\n',
 '    <species>Rhea americana</species>\n',
 '    <subspecies/>\n',
 '    <collections>Birds</collections>\n',
 '    <creation_year>2023</creation_year>\n',
 '    <absolute_url>https://portal.museumfuernaturkunde.berlin/detail/7d1264cca2e93009237e</absolute_url>\n',
 '    <copyright>MfN Berlin, https://ror.org/052d1a351</copyright>\n',
 '    <license>CC0 1.0 Creative Commons Public Domain Dedication</license>\n',
 '    <authors>MfN Digitization</authors>\n',
 '  </row>\n']

As mentioned, XML files can be validated. Validation can be carried out, for instance, using an XML Schema Definition (XSD). Below an example of how a snippet of the bird dataset in XML format can be validated using and XSD file using the Python lxml library and a simple function suggested by this [Stackoverflow question](https://stackoverflow.com/questions/299588/validating-with-an-xml-schema-in-python). 


In [36]:
# The structure of the XSD file
with open (os.path.join(data_path,"data.xsd")) as f:
    content = f.readlines()
content

['<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">\n',
 '  <xs:element name="data">\n',
 '    <xs:complexType>\n',
 '      <xs:sequence>\n',
 '        <xs:element name="row">\n',
 '          <xs:complexType>\n',
 '            <xs:sequence>\n',
 '              <xs:element name="catalogue_id" type="xs:string"/>\n',
 '              <xs:element name="key" type="xs:string"/>\n',
 '              <xs:element name="scientific_name" type="xs:string"/>\n',
 '              <xs:element name="title" type="xs:string"/>\n',
 '              <xs:element name="class" type="xs:string"/>\n',
 '              <xs:element name="family" type="xs:string"/>\n',
 '              <xs:element name="genus" type="xs:string"/>\n',
 '              <xs:element name="species" type="xs:string"/>\n',
 '              <xs:element name="subspecies" type="xs:string"/>\n',
 '              <xs:element name="collections" type="xs:string"/>\n',
 '              <xs:element name="creation_year" type="xs:integer"/>\n',
 '     

In [37]:
# Validate function

def validate(xml_path: str, xsd_path: str) -> bool:

    xmlschema_doc = etree.parse(xsd_path)
    xmlschema = etree.XMLSchema(xmlschema_doc)

    xml_doc = etree.parse(xml_path)
    result = xmlschema.validate(xml_doc)

    return result

In [38]:
# Validation of the XML data snippet containing only the first data row

if validate(os.path.join(data_path,"dataset_snippet.xml"), os.path.join(data_path,"data.xsd")):
    print('Valid!')
else:
    print('Not valid!')

Valid!


XML files can also be validated using a Document Type Definition, in short **DTD**. The DTD can be given as a separate file with .dtd extension or embedded in the XML file. The example considered here uses the same portion (first row) of the example dataset of bird specimens with an internal DTD.

In [39]:
# The structure of the XML with internal
with open (os.path.join(data_path,"dataset_snippet_with_dtd.xml")) as f:
    content = f.readlines()
content[0:20]

["<?xml version='1.0' encoding='utf-8'?>\n",
 '<!DOCTYPE data [\n',
 '        <!ELEMENT data (row)*>\n',
 '        <!ELEMENT row (catalogue_id|key|scientific_name|title|class|family|genus|species|subspecies|collections|creation_year|absolute_url|copyright|license|authors)*>\n',
 '        <!ELEMENT catalogue_id (#PCDATA)>\n',
 '        <!ELEMENT key (#PCDATA)>\n',
 '        <!ELEMENT scientific_name (#PCDATA)>\n',
 '        <!ELEMENT title (#PCDATA)>\n',
 '        <!ELEMENT class (#PCDATA)>\n',
 '        <!ELEMENT family (#PCDATA)>\n',
 '        <!ELEMENT genus (#PCDATA)>\n',
 '        <!ELEMENT species (#PCDATA)>\n',
 '        <!ELEMENT subspecies (#PCDATA)>\n',
 '        <!ELEMENT collections (#PCDATA)>\n',
 '        <!ELEMENT creation_year (#PCDATA)>\n',
 '        <!ELEMENT absolute_url (#PCDATA)>\n',
 '        <!ELEMENT copyright (#PCDATA)>\n',
 '        <!ELEMENT license (#PCDATA)>\n',
 '        <!ELEMENT authors (#PCDATA)>\n',
 '        ]>\n']

The DTD is inserted right after the XML prolog. It provides information on the legal elements and attributes of the XML document and values they take:
- ***!DOCTYPE data***: defines the root element of the XML document
- ***!ELEMENT row (catalogue_id|key|scientific_name|title|class|family|genus|species|subspecies|collections|creation_year|absolute_url|copyright|license|authors)***: defines the elements that the root must contain
- ***!ELEMENT catalogue_id (#PCDATA)***: defines that the element catalogue_id is of type parseable character data (PCDATA). Similar definitions are given for all the other elements that make up a row of data.



In [40]:
# The validity of the XML file is tested against the internal DTD.
# If the verification is successful (as in this case), there is no output,
# otherwise the parser returns an error message.
parser = etree.XMLParser(dtd_validation=True)
tree = etree.parse(os.path.join(data_path,"dataset_snippet_with_dtd.xml"), parser)

### 2.4: JSON <a class="anchor" id="json"></a>

A JSON (JavaScript Object Notation) data file consists of **objects** included within curly braces, **data** assigned as key-value pairs included within quotation marks if strings, left unquoted if numbers, and **arrays** included in square brackets. The various data elements in a JSON file are separated by **commas**.

This structure can be easily traced in the image below that reproduces the first row of the example dataset used in this Jupyter notebook. The key-value pair structure is here emphasized with colours. The keys are in light blue, while the values are in red if strings, green if numbers, and deep blue if nulls or boolean values. All the key-value pairs for a data row are enclosed within a set of curly braces and each key-value pair is separated from the successive by a comma. Each data row is also separated from the following by a comma (see last line in the image).

The square bracket opening at the beginning will close at the end of the file (not visible here), after the last object, i.e., the last data row, as the dataset is here reproduced as a list of JSON objects.

![Here](json_object.png)

The Python library Pandas allows to read datasets in the JSON file format easily. The data are read as a standard Pandas dataframe, not differently from the result when reading the same data in CSV or XML format. With the JSON data format, however, it is necessary to specify how the data objects should be read. In this case, it is specified that the json objects should be considered as the dataframe rows (this is the meaning of of the argument orient="records"). After the data have been read as a dataframe using the Pandas library, they can be easily analysed.

In [41]:
# Reading the JSON data file into a dataframe
df_json = pd.read_json(os.path.join(data_path, "dataset.json"), orient="records")

In [42]:
df_json

,catalogue_id,key,scientific_name,title (scientific name or common name),class,family,genus,species,subspecies,collections,creation_year,absolute_url,copyright,license,authors
0,ZMB_AVES_2000-14765,7d1264cca2e93009237e,Rhea americana,Rhea americana,Aves,Rheidae,Rhea,Rhea americana,None,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
1,ZMB_AVES_2000-14765,46e9071723ddbababd06,Rhea americana,Rhea americana,Aves,Rheidae,Rhea,Rhea americana,None,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
2,ZMB_AVES_2000-31350,cd50680edb26a356d7f1,Struthio camelus,Struthio camelus,Aves,Struthionidae,Struthio,Struthio camelus,None,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
3,ZMB_AVES_2000-31350,1af8bb8651519838f87c,Struthio camelus,Struthio camelus,Aves,Struthionidae,Struthio,Struthio camelus,None,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
4,ZMB_AVES_2000-31351,b2d8c2a150d82453d35c,Struthio molybdophanes,Struthio molybdophanes,Aves,Struthionidae,Struthio,Struthio molybdophanes,None,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13283,ZMB_Aves_9985,ca892e3d44beef9ca4dc,Psittacula krameri borealis,Psittacula krameri borealis,Aves,Psittacidae,Psittacula,Psittacula krameri,Psittacula krameri borealis,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
13284,ZMB_Aves_999,e76c936cca035eb8557d,Falco biarmicus tanypterus,Falco biarmicus tanypterus,Aves,Falconidae,Falco,Falco biarmicus,Falco biarmicus tanypterus,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
13285,ZMB_Aves_999,e5672b9636cec6d995e8,Falco biarmicus tanypterus,Falco biarmicus tanypterus,Aves,Falconidae,Falco,Falco biarmicus,Falco biarmicus tanypterus,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
13286,ZMB_Aves_9994,6ac8193d5913aa55de96,Psittacula longicauda longicauda,Psittacula longicauda longicauda,Aves,Psittacidae,Psittacula,Psittacula longicauda,Psittacula longicauda longicauda,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization


In many cases, however, for performance issues or other reasons, it might be preferable to read/write a JSON data file using the [native support for JSON in the Python programming language](https://docs.python.org/3/library/json.html) rather than via the Pandas library. 

In [43]:
# Reading in the JSON data file using Python json library
with open(os.path.join(data_path, "dataset.json"), "r") as f:
    json_data = json.load(f)

Printed out to screen one has the bare structure of the JSON file with its key-value pairs, as in the image displayed at the beginning of this section.

In [44]:
json_data[0:10]

[{'catalogue_id': 'ZMB_AVES_2000-14765',
  'key': '7d1264cca2e93009237e',
  'scientific_name': 'Rhea americana',
  'title (scientific name or common name)': 'Rhea americana',
  'class': 'Aves',
  'family': 'Rheidae',
  'genus': 'Rhea',
  'species': 'Rhea americana',
  'subspecies': None,
  'collections': 'Birds',
  'creation_year': 2023,
  'absolute_url': 'https://portal.museumfuernaturkunde.berlin/detail/7d1264cca2e93009237e',
  'copyright': 'MfN Berlin, https://ror.org/052d1a351',
  'license': 'CC0 1.0 Creative Commons Public Domain Dedication',
  'authors': 'MfN Digitization'},
 {'catalogue_id': 'ZMB_AVES_2000-14765',
  'key': '46e9071723ddbababd06',
  'scientific_name': 'Rhea americana',
  'title (scientific name or common name)': 'Rhea americana',
  'class': 'Aves',
  'family': 'Rheidae',
  'genus': 'Rhea',
  'species': 'Rhea americana',
  'subspecies': None,
  'collections': 'Birds',
  'creation_year': 2023,
  'absolute_url': 'https://portal.museumfuernaturkunde.berlin/detail/46e

In [45]:
# As the JSON data file is a list of JSON objects, once the file is read in memory, a subset of data points can be selected
# using standard Python list syntax.

json_data[0:1] # First data point

[{'catalogue_id': 'ZMB_AVES_2000-14765',
  'key': '7d1264cca2e93009237e',
  'scientific_name': 'Rhea americana',
  'title (scientific name or common name)': 'Rhea americana',
  'class': 'Aves',
  'family': 'Rheidae',
  'genus': 'Rhea',
  'species': 'Rhea americana',
  'subspecies': None,
  'collections': 'Birds',
  'creation_year': 2023,
  'absolute_url': 'https://portal.museumfuernaturkunde.berlin/detail/7d1264cca2e93009237e',
  'copyright': 'MfN Berlin, https://ror.org/052d1a351',
  'license': 'CC0 1.0 Creative Commons Public Domain Dedication',
  'authors': 'MfN Digitization'}]

Extracting information from the JSON data file without passing through the dataframe form is simple. For instance, a solution to extract all the unique values of the catalogue_id key is:

In [46]:
cat_id_list = [] # Creating an empty list to append the catalogue_id values

for el in json_data: # looping through the json data object to extract the values of the key "catalogue_id"
    cat_id_list.append(el["catalogue_id"])

unique_cat_id_list = list(set(cat_id_list)) # the values extracted are not all unique as there are two or more images
                                            # for each bird specimens.
                                            # The unique values can be extracted using the using the set operation on the list

print(len(unique_cat_id_list)) # unique identifiers in the dataset

6628


All the other pieces of information can be extracted from the json data object in similar fashion.

### 2.5: RDF <a class="anchor" id="rdf"></a>

The Resource Description Framework (RDF) is a standard for data interchange on the web. RDF files allow to exchange data using a triplet syntax (subject-predicate-object) that can be turned into a directed graph. Each triplet element is identified by a URI (Uniform Resource Identifier). We will quickly examine the properties of RDF files using an RDF version of the [*Breeding Bird Atlases*](https://catalog.data.gov/dataset/breeding-bird-atlases) dataset available on DATA.GOV and the Python library [rdflib](https://rdflib.readthedocs.io/en/stable/).

In [47]:
g = Graph() # Graph object instantiated
g.parse("https://data.ny.gov/api/views/vk8g-ypxi/rows.rdf?accessType=DOWNLOAD") # rdf data file parsed

<Graph identifier=Nd9c9c38964404befbbf23ad9f245e366 (<class 'rdflib.graph.Graph'>)>

As evident from the result of the code output, the data are now held in a graph object. The length of the graph object is the number of triplets in the graph.

In [48]:
print(len(g))

7578


The URIs associated to the elements of the triplets can be printed out:

In [49]:
for uri in g[0:3]:
    pprint.pprint(uri)

The graph object can also be queried to extract only the predicates of the the triples (which are based on the syntax subject-predicate-object, as described above).

In [51]:
predicate_query = g.query("""
                     select ?predicates
                     where {?s ?predicates ?o}
                     """)

for row in predicate_query:
    print('%s' % row)

https://data.ny.gov/resource/vk8g-ypxi/scientific_name
https://data.ny.gov/resource/vk8g-ypxi/county
http://www.socrata.com/rdf/terms#rowID
https://data.ny.gov/resource/vk8g-ypxi/year
https://data.ny.gov/resource/vk8g-ypxi/family_name
https://data.ny.gov/resource/vk8g-ypxi/day
https://data.ny.gov/resource/vk8g-ypxi/year
http://xmlns.com/foaf/0.1/document
https://data.ny.gov/resource/vk8g-ypxi/family_description
https://data.ny.gov/resource/vk8g-ypxi/fed_region
http://www.w3.org/2000/01/rdf-schema#member
https://data.ny.gov/resource/vk8g-ypxi/county
https://data.ny.gov/resource/vk8g-ypxi/month
https://data.ny.gov/resource/vk8g-ypxi/day
https://data.ny.gov/resource/vk8g-ypxi/month
https://data.ny.gov/resource/vk8g-ypxi/breeding_status
https://data.ny.gov/resource/vk8g-ypxi/scientific_name
https://data.ny.gov/resource/vk8g-ypxi/family_description
https://data.ny.gov/resource/vk8g-ypxi/block
https://data.ny.gov/resource/vk8g-ypxi/county
http://xmlns.com/foaf/0.1/document
https://data.ny.go

The graph structure of the RDF file can be visualised graphically. The RDF dataset is way too complex to allow in full for a meaningful graphical representation, but below you can find an image of the graph generated considering only the very first portion of the dataset and the code displayed below. 
![Here](rdf_graph.png)

In [52]:
import io
import pydotplus
from IPython.display import display, Image
from rdflib.tools.rdf2dot import rdf2dot

def visualize(g):
    stream = io.StringIO()
    rdf2dot(g, stream, opts = {display})
    dg = pydotplus.graph_from_dot_data(stream.getvalue())
    png = dg.create_png()
    display(Image(png))

### 2.6: Parquet <a class="anchor" id="parquet"></a>

PARQUET files store data column-based rather than row-based. This is not immediately evident when reading parquet files using the Pandas library because the result is always a  Pandas dataframe.
<br>
Here there is an example using the dataset of the bird mounted specimens:

In [53]:
df_csv = pd.read_csv(os.path.join(data_path, "dataset.csv"))
df_csv.head(2)

,catalogue_id,key,scientific_name,title (scientific name or common name),class,family,genus,species,subspecies,collections,creation_year,absolute_url,copyright,license,authors
0,ZMB_AVES_2000-14765,7d1264cca2e93009237e,Rhea americana,Rhea americana,Aves,Rheidae,Rhea,Rhea americana,NaN,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
1,ZMB_AVES_2000-14765,46e9071723ddbababd06,Rhea americana,Rhea americana,Aves,Rheidae,Rhea,Rhea americana,NaN,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization


In [54]:
df_parquet = pd.read_parquet(os.path.join(data_path, "dataset.parquet"), engine='fastparquet')
df_parquet.head(2)

,catalogue_id,key,scientific_name,title (scientific name or common name),class,family,genus,species,subspecies,collections,creation_year,absolute_url,copyright,license,authors
0,ZMB_AVES_2000-14765,7d1264cca2e93009237e,Rhea americana,Rhea americana,Aves,Rheidae,Rhea,Rhea americana,None,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization
1,ZMB_AVES_2000-14765,46e9071723ddbababd06,Rhea americana,Rhea americana,Aves,Rheidae,Rhea,Rhea americana,None,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization


In [55]:
%%time
val = "8dd2e0623b1caa21c461"
df_csv.query("key == @val")

CPU times: user 3.87 ms, sys: 632 µs, total: 4.5 ms
Wall time: 4.05 ms


,catalogue_id,key,scientific_name,title (scientific name or common name),class,family,genus,species,subspecies,collections,creation_year,absolute_url,copyright,license,authors
8005,ZMB_Aves_2000-75388,8dd2e0623b1caa21c461,NaN,Schwalbentaube,Aves,Columbidae,NaN,NaN,NaN,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization


In [56]:
%%time 
df_parquet.query("key == @val")

CPU times: user 3.8 ms, sys: 905 µs, total: 4.71 ms
Wall time: 3.97 ms


,catalogue_id,key,scientific_name,title (scientific name or common name),class,family,genus,species,subspecies,collections,creation_year,absolute_url,copyright,license,authors
8005,ZMB_Aves_2000-75388,8dd2e0623b1caa21c461,None,Schwalbentaube,Aves,Columbidae,None,None,None,Birds,2023,https://portal.museumfuernaturkunde.berlin/det...,"MfN Berlin, https://ror.org/052d1a351",CC0 1.0 Creative Commons Public Domain Dedication,MfN Digitization


Performance difference at query time between the CSV and the PARQUET dataset is not noticeable in this specific case. The CSV format would rather be a better choice. The dataset is still too small to make the use of PARQUET meaningful and the PARQUET version lacks associated metadata that can be used to filter and reduce response at query time.
When working with big data that require specific formats like PARQUET, it is recommended to use the Python library [DASK](https://www.dask.org), which allows for parallel computations.

---